# Algorytmy macierzowe
## lab 2

In [96]:
import numpy as np
from time import time
from matplotlib import pyplot as plt

Własna klasa Number do zliczania ilości operacji

In [245]:
class Number(float):
    operation_counter = 0
    
    def __repr__(self) -> str:
        return f"{self:.8f}"
    
    def __radd__(self, other):
        Number.operation_counter += 1
        return Number(super().__radd__(other))

    def __add__(self, other):
        Number.operation_counter += 1
        return Number(super().__add__(other))
    
    def __rsub__(self, other):
        Number.operation_counter += 1
        return Number(super().__rsub__(other))

    def __sub__(self, other):
        Number.operation_counter += 1
        return Number(super().__sub__(other))
    
    def __mul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__mul__(other))
    
    def __rmul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__rmul__(other))
    
    def __truediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__truediv__(other))
    
    def __rtruediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__rtruediv__(other))
    
    def counter_reset():
        Number.operation_counter = 0

Generowanie macierzy

In [246]:
def random_matrix(matrix_size, min_val, max_val):
    """Return matrix with random Number floats from [min_val, max_val)"""
    matrix = (max_val - min_val) * np.random.random(matrix_size) + min_val
    return np.array([[Number(value) for value in row] for row in matrix], dtype=Number)

def transform_to_float(A):
    """np.linalg.inv doesnt work with dtype=Number"""
    return np.array(A, dtype=float)

In [306]:
exp = 1
matrix_size = (2**exp, 2**exp)

min_val = 0.00000001
max_val = 1

A = random_matrix(matrix_size, min_val, max_val)

### Rekurencyjne odwracanie macierzy

In [307]:
def inverse(A):
    if A.shape[0] != A.shape[1]:
        print("ERROR: Wrong matrix size!")
        return None
    
    if len(A) == 1:
        return A
    
    if len(A) == 2:
        return np.array(
            [[A[1,1], -A[0,1]],
             [-A[1, 0], A[0,0]]]
        ) / (A[0,0]*A[1,1] - A[0,1]*A[1,0])
        
    matrix_size = len(A)
    A11 = A[:matrix_size//2, :matrix_size//2]
    A12 = A[:matrix_size//2, matrix_size//2:]
    A21 = A[matrix_size//2:, :matrix_size//2]
    A22 = A[matrix_size//2:, matrix_size//2:]
    
    A11_inv = inverse(A11)
    S22 = A22 - A21 @ A11_inv @ A12
    S22_inv = inverse(S22)
    B11 = A11_inv @ (np.identity(len(A11)) + A12 @ S22_inv @ A21 @ A11_inv)
    B12 = -A11_inv @ A12 @ S22_inv
    B21 = -S22_inv @ A21 @ A11_inv
    B22 = S22_inv
    
    return np.vstack((np.hstack((B11, B12)), np.hstack((B21, B22))))

In [308]:
Number.counter_reset()
res = inverse(A)

print(f"Matrix size: {matrix_size}")
print("Is correct?:", np.allclose(transform_to_float(res), np.linalg.inv(transform_to_float(A))))
print("Number of operations:", Number.operation_counter)

Matrix size: (2, 2)
Is correct?: True
Number of operations: 3


### Rekurencyjna LU faktoryzacja

In [316]:
def LU_factorization(A):
    if A.shape[0] != A.shape[1]:
        print("ERROR: Wrong matrix size!")
        return None
    
    if len(A) == 1:
        L = np.array([[Number(1.0)]], dtype=Number)
        U = A.copy()
        return L, U
    
    matrix_size = len(A)
    A11 = A[:matrix_size//2, :matrix_size//2]
    A12 = A[:matrix_size//2, matrix_size//2:]
    A21 = A[matrix_size//2:, :matrix_size//2]
    A22 = A[matrix_size//2:, matrix_size//2:]
    
    L11, U11 = LU_factorization(A11)
    U11_inv = inverse(U11)
    L21 = A21 @ U11_inv
    L11_inv = inverse(L11)
    U12 = L11_inv @ A12
    S = A22 - A21 @ U11_inv @ L11_inv @ A12
    LS, US = LU_factorization(S)
    
    L = np.vstack((np.hstack((L11, np.zeros(L11.shape))), np.hstack((L21, LS))))
    U = np.vstack((np.hstack((U11, U12)), np.hstack((np.zeros(L11.shape), US))))
    return L, U

In [317]:
Number.counter_reset()
L, U = LU_factorization(A)

# print(A)
# print()
# print(L @ U)
# print()

print(f"Matrix size: {matrix_size}")
print("Is correct?:", np.allclose(transform_to_float(A), transform_to_float(L @ U)))
print(np.equal(transform_to_float(A), transform_to_float(L @ U)))
print("Number of operations:", Number.operation_counter)

import scipy

x = scipy.linalg.lu(transform_to_float(A))
print()
print(x[1])
print(L)
print()
print(x[2])
print(U)

print()
print(A)
print(x[1] @ x[2])
print(L @ U)



Matrix size: (2, 2)
Is correct?: False
[[ True  True]
 [False  True]]
Number of operations: 28

[[1.        0.       ]
 [0.1276774 1.       ]]
[[1.00000000 0.0]
 [0.11568821 1.00000000]]

[[0.95189171 0.65444784]
 [0.         0.58077036]]
[[0.12153506 0.66432855]
 [0.0 0.57759286]]

[[0.12153506 0.66432855]
 [0.95189171 0.65444784]]
[[0.95189171 0.65444784]
 [0.12153506 0.66432855]]
[[0.12153506 0.66432855]
 [0.01406017 0.65444784]]


### Rekurencyjne obliczanie wyznacznika